In [0]:
import tensorflow as tf
import os
import numpy as np

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt','https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


In [0]:
# from google.colab import files
# path_to_file = list(files.upload().keys())[0]

In [0]:
text = open(path_to_file,'rb').read().decode(encoding='utf-8')

In [0]:
vocab = sorted(set(text))
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_int(text): return np.array([char2idx[c] for c in text])

In [13]:
print("TEXT: ",text[:20])
print("ENCODED: ",text_int(text[:20]))

TEXT:  First Citizen:
Befor
ENCODED:  [18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56]


In [15]:
# def int_text(ints):
#   try:
#     ints = ints.numpy()
#   except:
#     pass
  # return ''.join(idx2char[ints])
def int_text(ints): return ''.join(idx2char[ints])

print(int_text(text_int(text[:20])))

First Citizen:
Befor


In [0]:
seq_length =100
examples_per_epoch = len(text)//(seq_length+1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_int(text))

In [0]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [0]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target())

In [0]:
batch_size = 64
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024
buffer_size = 10000
data = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)

In [27]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
                               tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
                               tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
                               tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(vocab_size, embedding_dim, rnn_units, batch_size)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
lstm_1 (LSTM)                (64, None, 1024)          5246976   
_________________________________________________________________
dense_1 (Dense)              (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [28]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [33]:
print(len(example_batch_predictions))
print(example_batch_predictions)

pred = example_batch_predictions[0]
print(len(pred))
print(pred)

time_pred = pred[0]
print(len(time_pred))
print(time_pred)

sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)

64
tf.Tensor(
[[[ 1.62878609e-03  4.86297067e-03  2.51444941e-03 ... -1.35758182e-03
    2.37798179e-03 -6.24486897e-03]
  [-2.10060272e-03  9.32632945e-03  3.01539549e-03 ... -8.80539417e-03
    1.48707535e-04 -3.34495050e-03]
  [ 6.75477227e-03  2.69828341e-03  1.52128970e-03 ... -2.02095928e-03
   -8.23725015e-04 -6.27625547e-03]
  ...
  [ 8.99430364e-04  4.71661333e-03  5.10351965e-03 ... -5.21392468e-03
   -3.24455672e-04 -6.20030006e-03]
  [ 9.09179635e-03 -4.49290965e-04  3.32246581e-03 ...  1.18604198e-03
   -1.52763212e-03 -8.69839359e-03]
  [ 5.84403891e-03 -1.21804141e-03  3.25670023e-03 ... -1.41509401e-03
    2.48032017e-03 -8.27911124e-03]]

 [[ 5.08736540e-03 -2.72444473e-03  3.57016525e-03 ...  1.93215499e-03
   -2.69525335e-03  6.00479171e-03]
  [ 2.82403245e-03 -5.35125146e-03  1.89822773e-03 ... -2.87961657e-03
    2.01648776e-03  1.06530508e-03]
  [ 3.40052322e-03 -7.33436598e-03 -7.07694096e-04 ... -5.22416364e-03
    1.09854178e-03  3.32423090e-03]
  ...
  [ 4.278

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [35]:
history = model.fit(data, epochs=50, callbacks=[checkpoint_callback])

Epoch 1/50
172/172 [==============================] - 9s 53ms/step - loss: 2.5955
Epoch 2/50
172/172 [==============================] - 9s 53ms/step - loss: 1.8945
Epoch 3/50
172/172 [==============================] - 9s 53ms/step - loss: 1.6424
Epoch 4/50
172/172 [==============================] - 9s 53ms/step - loss: 1.5060
Epoch 5/50
172/172 [==============================] - 9s 53ms/step - loss: 1.4246
Epoch 6/50
172/172 [==============================] - 9s 53ms/step - loss: 1.3687
Epoch 7/50
172/172 [==============================] - 9s 53ms/step - loss: 1.3232
Epoch 8/50
172/172 [==============================] - 9s 53ms/step - loss: 1.2839
Epoch 9/50
172/172 [==============================] - 9s 53ms/step - loss: 1.2474
Epoch 10/50
172/172 [==============================] - 9s 53ms/step - loss: 1.2135
Epoch 11/50
172/172 [==============================] - 9s 53ms/step - loss: 1.1792
Epoch 12/50
172/172 [==============================] - 9s 53ms/step - loss: 1.1423
Epoch 13/50
1

NameError: ignored

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

In [0]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [41]:
checkpoint_num = 10
model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_" + str(checkpoint_num)))
model.build(tf.TensorShape([1, None]))

AttributeError: ignored

In [0]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [44]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: romeo
romeour
Art tall'd with virtuous that ride so state
And bitter than theyer.

ROMEO:
If I may trust, if God f pity give us becomes my mistress' head;
The knightful block news, wanting, after thoughts,
I'll mend it much dis sendency to oppenith,
In peeping hot the letter with the goast
Which eyes the wolf.

DUKE VINCENTIO:
By heavens, I pray thee, gove a corple, but that
scarce for your nept way to-nir--

First Citizen:
I the great cousin, ever.

VOLUMNIA:
O, good my lord, despite; I must forget it,
Rather than honour of himself, hid
From these old Gloucester's pardon.

MENENIUS:
Sir, that's oppress:
Therefore accompand us with well-spoken, and none suspected.

BIONDELLO:
Why, is it not now and now you are
To shrie king and stends already; and in the
sequever than at home by the evil diet and
dus
